# Homework Week 1

### Question 1. Understanding docker first run
Run docker with the python:3.12.8 image in an interactive mode, use the entrypoint bash.

What's the version of pip in the image?

In [ ]:
docker run -it --entrypoint bash python:3.12.8
pip --version
# pip 24.3.1 from /usr/local/lib/python3.12/site-packages/pip (python 3.12)

ANSWER: 24.3.1

### Prepare Postgres
- Modified script to ingest green_taxi data
- Modified dockerfile to include green_taxi script
- docker build -t yellow-taxi-pipeline:v5 .
- docker compose up -d --build
- docker compose run --rm data-loader python scripts/pipeline-green.py /app/data/green_tripdata_2019-10.csv

### Question 2. Understanding Docker networking and docker-compose
Given the following docker-compose.yaml, what is the hostname and port that pgadmin should use to connect to the postgres database?

ANSWER: postgres:5432

### Question 3. Trip Segmentation Count
During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

Up to 1 mile
In between 1 (exclusive) and 3 miles (inclusive),
In between 3 (exclusive) and 7 miles (inclusive),
In between 7 (exclusive) and 10 miles (inclusive),
Over 10 miles

In [ ]:
WITH TripCategories AS (
    SELECT
        CASE
            WHEN trip_distance <= 1 THEN 'Up to 1 mile'
            WHEN trip_distance > 1 AND trip_distance <= 3 THEN '1-3 miles'
            WHEN trip_distance > 3 AND trip_distance <= 7 THEN '3-7 miles'
            WHEN trip_distance > 7 AND trip_distance <= 10 THEN '7-10 miles'
            WHEN trip_distance > 10 THEN 'Over 10 miles'
            ELSE 'Unknown' -- Handle cases where trip_distance is NULL or invalid
        END as trip_category
    FROM
        public.green_taxi_data
    WHERE
        lpep_pickup_datetime >= '2019-10-01' AND lpep_pickup_datetime < '2019-11-01'
)
SELECT
    trip_category,
    COUNT(*) as trip_count
FROM
    TripCategories
GROUP BY
    trip_category
ORDER BY
    CASE trip_category
        WHEN 'Up to 1 mile' THEN 1
        WHEN '1-3 miles' THEN 2
        WHEN '3-7 miles' THEN 3
        WHEN '7-10 miles' THEN 4
        WHEN 'Over 10 miles' THEN 5
        ELSE 6 -- Ensure 'Unknown' category is last
    END;

ANSWER: 104,802; 198,924; 109,603; 27,678; 35,189

### Question 4. Longest trip for each day
Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

Tip: For every day, we only care about one single trip with the longest distance.

In [ ]:
WITH DailyMaxTrips AS (
    SELECT
        DATE(lpep_pickup_datetime) AS pickup_day,
        MAX(trip_distance) AS max_trip_distance
    FROM
        public.green_taxi_data
    GROUP BY
        pickup_day
)
SELECT
    pickup_day
FROM
    DailyMaxTrips
ORDER BY
    max_trip_distance DESC
LIMIT 1;

ANSWER: 2019-10-31

### Question 5. Three biggest pickup zones
Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

Consider only lpep_pickup_datetime when filtering by date.

East Harlem North, East Harlem South, Morningside Heights
East Harlem North, Morningside Heights
Morningside Heights, Astoria Park, East Harlem South
Bedford, East Harlem North, Astoria Park

In [ ]:
SELECT
    CONCAT(zpu."Borough", ' / ', zpu."Zone") AS "pickup_loc"
FROM
    public.green_taxi_data t
JOIN
    public.zones zpu ON t."PULocationID" = zpu."LocationID"
WHERE
    DATE(t.lpep_pickup_datetime) = '2019-10-18'
GROUP BY
    pickup_loc
HAVING
    SUM(t.total_amount) > 13000
ORDER BY
    SUM(t.total_amount) DESC
LIMIT 3;

ANSWER: East Harlem North, East Harlem South, Morningside Heights

### Question 6. Largest tip
For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?

Note: it's tip , not trip

We need the name of the zone, not the ID.

Yorkville West
JFK Airport
East Harlem North
East Harlem South

In [ ]:
SELECT
    CONCAT(zdo."Borough", ' / ', zdo."Zone") AS dropoff_zone
FROM
    public.green_taxi_data t
JOIN
    public.zones zpu ON t."PULocationID" = zpu."LocationID"
JOIN
    public.zones zdo ON t."DOLocationID" = zdo."LocationID"
WHERE
    zpu."Zone" = 'East Harlem North'
    AND DATE(t.lpep_pickup_datetime) BETWEEN '2019-10-01' AND '2019-10-31'
ORDER BY
    t.tip_amount DESC
LIMIT 1;

ANSWER: JFK Airport

### Question 7. Terraform Workflow
Which of the following sequences, respectively, describes the workflow for:

Downloading the provider plugins and setting up backend,
Generating proposed changes and auto-executing the plan
Remove all resources managed by terraform`
Answers:

terraform import, terraform apply -y, terraform destroy
teraform init, terraform plan -auto-apply, terraform rm
terraform init, terraform run -auto-approve, terraform destroy
terraform init, terraform apply -auto-approve, terraform destroy
terraform import, terraform apply -y, terraform rm

ANSWER: terraform init, terraform apply -auto-approve, terraform destroy